# Agentic AI

## Prerequisites

### Install a Local LLM with Ollama

To run this project locally, we will install and use **Ollama**, a lightweight runtime for local large language models.

**Download Ollama:**  
https://ollama.com/

Once installed, you can pull any model you want to run.  
Below are a few recommended examples, but you are free to pick any size or model from the Ollama library.

ollama pull qwen3:0.6b

or

ollama pull ibm/granite4:350m

or

Choose any model you prefer, make sure the model supports tools.
Browse available models here:
https://ollama.com/library



### Python requirements

In [1]:
!pip install langgraph langchain-google-genai langchain-core mcp langchain-ollama

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/718.4 kB ? eta -:--:--
   -------------- ------------------------- 262.1/718.4 kB ? eta -:--:--
   ---------------------------------------- 718.4/718.4 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.5 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.5 MB 2.4 MB/s eta 0:00:02
   -------------- ------------------------- 1.3/3.5 MB 2.3 MB/s eta 0:00:01
   -------------------- ------------------- 1.8/3.5 MB 2.3 MB/s eta 0:00:01
   -------------------------- ------------- 2.4/3.5 MB 2.3 MB/s eta 0:00:01
   ----------------------------- ---------- 2.6/3.5 MB 2.3 MB/s eta 0:00:01
   ----------------------------------- ---- 3.1/3.5 MB 2.3 MB/s eta 0:00:01
   -------------------------------------- - 3.4/3.5 MB 2.3 MB/s eta 0:00:01
   -------------------------


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\bhaah\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 1. Define FastMCP Tools

## 1.1 Define the heuristic search functions 
we choose the first assignment:

In [20]:
#--------------------------- color block ---------------------------
goal_state = []
def init_goal_for_search(goal_blocks):
    currNumber = ""
    global goal_state
    goal_state=[]
    counter = 0
    for c in goal_blocks:
        # print(c,'to add on ', currNumber, 'and we are on the last index ', goal_blocks.index(c),' and the length is ', len(goal_blocks))
        if counter == len(goal_blocks) - 1:
            # print('we are on the last character', c , '-to add on ', currNumber)
            currNumber += c
            goal_state.append(int(currNumber))
            continue
        if c==','  :
            goal_state.append(int(currNumber))
            currNumber = ""
        else:
            currNumber += c
        counter += 1
    for goal in goal_state:
        # pass
        print('goal state after init:', goal)

class color_blocks_state:
    # you can add global params
    
    def __init__(self, blocks_str, **kwargs):
        # you can use the init function for several purposes
        self.blocks = []
        currNum = ""
        currTuple = ()
        for c in blocks_str:
            if c == ',' and currNum:
                currTuple += (int(currNum),)
                currNum = ""
                continue
            if c=='(' :
                currNum = ""
                currTuple = ()
                continue
            if c==')' :
                currTuple += (int(currNum),)
                currNum = ""
                self.blocks.append(currTuple)
                currTuple = ()
                continue
            currNum += c
        

    @staticmethod
    def is_goal_state(_color_blocks_state):
        # print('goal_state:', goal_state)
        for i in range(len(goal_state)):
            # print(_color_blocks_state.blocks[i][0], ',', goal_state[i])
            if _color_blocks_state.blocks[i][0] != goal_state[i]:
                return False
        return True
    
    def get_neighbors(self):
        neighbors = []
        for i in range (len(self.blocks)):
            new_state_flip = self.flip_block(i)
            # if i!= len(self.blocks):
            neighbors.append((new_state_flip,1))
        for i in range (len(self.blocks)):
            new_state_spin = self.spin_block(i)
            neighbors.append((new_state_spin,1))
        return neighbors 
    # you can change the body of the function if you want
    # def __hash__(self):
    # you can change the body of the function if you want
    # def __eq__(self, other):
    # you can change the body of the function if you want

    def __hash__(self):
        return hash(tuple(self.blocks))

    def __eq__(self, other):
        return isinstance(other, color_blocks_state) and self.blocks == other.blocks
    # for debugging states
    def get_state_str(self):
        res = ""
        for b in self.blocks:
            res += "("
            for i in range(len(b)):
                res += str(b[i])
                if i != len(b) - 1:
                    res += ","
            res += ")"
            if b != self.blocks[-1]:
                res += ","
        return res

    def setBlocks(self, new_blocks):
        self.blocks = new_blocks

    #you can add helper functions [(1,2)]
    def flip_block(self, block_idx):
        # make a shallow copy of the list of tuples (safe!)
        copy_blocks = self.blocks.copy()

        # reverse everything from block_idx to the end
        copy_blocks[block_idx:] = reversed(copy_blocks[block_idx:])

        new_state = color_blocks_state("")
        new_state.setBlocks(copy_blocks)
        return new_state

    
    def spin_block(self, block_idx):
        copy_blocks = self.blocks.copy()

        copy_blocks[block_idx] = tuple(reversed(copy_blocks[block_idx]))

        new_state = color_blocks_state("")
        new_state.setBlocks(copy_blocks)
        return new_state

    def getBlockAt(self, index):
        return self.blocks[index]
    def getBlocks(self):
        return self.blocks

#--------------------------- search node ---------------------------

class search_node():
    def __init__(self, state, g=0, h=0, prev=None):
        self.state = state
        self.g = g
        self.h = h
        self.f = g + h
        self.prev = prev

    
    def setH(self, h):
        self.h = h
        self.f = self.g + self.h
    def setPrev(self, prev):
        self.prev = prev
    def __lt__(self, other):
        return (self.f < other.f) or (self.f == other.f and self.h < other.h)

    def get_neighbors(self):
        return self.state.get_neighbors()



#--------------------------- search ---------------------------
import heapq

import time



import bisect
from collections import Counter

# -----------------------------
# Optimized Open/Closed Sets
# -----------------------------
def create_open_set():
    """Return an empty open set (heap, dict)."""
    return [], {}  # heap, dict

def create_closed_set():
    """Return an empty closed set (dict)."""
    return {}  # state -> search_node

def add_to_open(node, open_set):
    """Add a node to the open set (heap + dict)."""
    open_heap, open_dict = open_set
    heapq.heappush(open_heap, node)
    open_dict[node.state] = node

def open_not_empty(open_set):
    open_heap, _ = open_set
    return len(open_heap) > 0

def get_best(open_set):
    """Pop the best node (lowest f) from the heap that is still in open_dict."""
    open_heap, open_dict = open_set
    while open_heap:
        node = heapq.heappop(open_heap)
        if node.state in open_dict and open_dict[node.state] is node:
            del open_dict[node.state]
            return node
    return None

def add_to_closed(node, closed_set):
    closed_set[node.state] = node

def duplicate_in_open(node, open_set):
    """Check if a better node already exists in open set."""
    _, open_dict = open_set
    existing = open_dict.get(node.state)
    if existing is None:
        return False
    if existing.g <= node.g:
        return True
    else:
        open_dict[node.state] = node
        return False

def duplicate_in_closed(node, closed_set):
    """Check if a better node already exists in closed set."""
    existing = closed_set.get(node.state)
    if existing is None:
        return False
    if existing.g <= node.g:
        return True
    else:
        del closed_set[node.state]
        return False

# -----------------------------
# Optimized A* Search
# -----------------------------
def search(start_state, heuristic, debug=False):
    """
    Optimized A* search using:
      - open set: (heap, dict)
      - closed set: dict
    """
    open_set = create_open_set()
    closed_set = create_closed_set()

    start_node = search_node(start_state, 0, heuristic(start_state))
    add_to_open(start_node, open_set)

    while open_not_empty(open_set):
        current = get_best(open_set)
        if current is None:
            break

        # if debug:
        # print(f"BEST: {current.state.get_state_str()}, g={current.g}, h={current.h}, f={current.f}")
        # print(f"Open size: {len(open_set[0])}, Closed size: {len(closed_set)}")

        # Goal check
        if color_blocks_state.is_goal_state(current.state):
            path = []
            node = current
            while node is not None:
                path.append(node)
                node = node.prev
            path.reverse()
            return path

        add_to_closed(current, closed_set)

        # Expand neighbors
        for neighbor_state, cost in current.get_neighbors():
            new_g = current.g + cost
            new_node = search_node(neighbor_state, new_g, heuristic(neighbor_state), current)

            if duplicate_in_open(new_node, open_set):
                continue
            if duplicate_in_closed(new_node, closed_set):
                continue

            add_to_open(new_node, open_set)

    return None  # No solution found

#--------------------------- heu ---------------------------
#--------------------------- heu ---------------------------

# --- Global Data ---
goal_state = []
index_map = {}       # Will map value -> list of indices [i1, i2...]
neighbors_map = {}   # Used for neighbor lookups
goal_adjacent_color_pairs = set() # Optimized set for base_heuristic

def init_goal_for_heuristics(goal_blocks):
    global goal_state, index_map, neighbors_map, goal_adjacent_color_pairs

    # 1. Parse Goal String
    goal_state = []
    curr = ""
    for c in goal_blocks:
        if c == ",":
            if curr:
                goal_state.append(int(curr))
            curr = ""
        else:
            curr += c
    if curr:
        goal_state.append(int(curr))

    # 2. Build index_map (value -> LIST of indices)
    # FIX: Initialize as lists to allow iteration in advanced_heuristic
    index_map = {}
    for i, v in enumerate(goal_state):
        if v not in index_map:
            index_map[v] = []
        index_map[v].append(i)

    # 3. Build neighbors_map (value -> sorted list of neighbors)
    neighbors_map = {}
    for i, v in enumerate(goal_state):
        neigh = []
        if i > 0:
            neigh.append(goal_state[i - 1])
        if i < len(goal_state) - 1:
            neigh.append(goal_state[i + 1])
        neighbors_map[v] = sorted(neigh)

    # 4. Build optimized pairs set for base_heuristic
    # (Matches logic from your previous code)
    pairs = set()
    for i in range(len(goal_state) - 1):
        left, right = goal_state[i], goal_state[i+1]
        if left <= right:
            pairs.add((left, right))
        else:
            pairs.add((right, left))
    goal_adjacent_color_pairs = pairs
    
def base_heuristic(_color_blocks_state):
    # print(type(_color_blocks_state))
    # newTemp = color_blocks_state("")
    # newTemp.setBlocks(_color_blocks_state)
    # _color_blocks_state = newTemp
    heu = 0
    # print('--->', _color_blocks_state.get_state_str())
    for i in range(len(_color_blocks_state.getBlocks())-1):
        currTupA = _color_blocks_state.getBlockAt(i)
        currTupB = _color_blocks_state.getBlockAt(i+1)
        currHue = 0 
        for j in range(len(goal_state)):
            if goal_state[j] in currTupA:
                # print('found block', goal_state[j], 'in', currTupA)
                currHue = simple_heuristic( currTupB, goal_state,j)
                if currHue == 0:
                    break
            if goal_state[j] in currTupB:
                # print('found block', goal_state[j], 'in', currTupB)
                currHue = simple_heuristic( currTupA, goal_state,j)
                if currHue == 0:
                    break
                
        # print
        heu += currHue    
        # print(f"Block {i} and Block {i+1} heuristic: {currHue} add to total {heu}")
            
    # print("heuristic:", heu)
    return heu   


def simple_heuristic( currTupB, goal_state,j):
    heuLocal = 0
    # print('checking for goal block:', goal_state[j], 'with current block:', currTupB)
    if j < len(goal_state)-1 and j!=0:
            if goal_state[j+1] in currTupB or goal_state[j-1] in currTupB:
                heuLocal+=0
            else:
                heuLocal +=1
    if j == 0:
        if goal_state[j+1] in currTupB:
            heuLocal+=0
        else:
            heuLocal +=1
    if j == len(goal_state)-1:
        if goal_state[j-1] in currTupB:
            heuLocal+=0
        else:
            heuLocal +=1
    return heuLocal


    
    
    
    
    

def advanced_heuristic(state):
    """
    Advanced heuristic checking index distances and specific edge cases.
    Fixed to handle index_map as a dictionary of lists.
    """
    blocks = state.getBlocks()
    h = 0
    
    if not blocks:
        return 0

    # Case 1: First block's left color check
    first_left = blocks[0][0]
    # Check if the exact color exists in the goal map
    found_first_left = first_left in index_map

    for i in range(len(blocks) - 1):
        A = blocks[i]
        B = blocks[i + 1]
        
        # Unpack tuple (v, h)
        # Note: Depending on your block structure, ensure A is (val, val) or similar
        # Assuming blocks are tuples like (color1, color2)
        
        # Case 2: Next block's left color availability
        b_left = B[0]
        b_in_goal = b_left in index_map

        # Case 3: Adjacency Logic using Index Map
        pair_ok = False

        for cA in A:
            if cA not in index_map:
                continue

            for cB in B:
                if cB not in index_map:
                    continue

                # FIX: Now index_map[cA] is a list, so we can iterate
                for idxA in index_map[cA]:
                    for idxB in index_map[cB]:
                        # Check if they are adjacent in the goal (diff is 1)
                        if abs(idxA - idxB) == 1:
                            pair_ok = True
                            break
                    if pair_ok: break
                if pair_ok: break
            if pair_ok: break

        # Penalties
        if not pair_ok:
            h += 1

        if not b_in_goal:
            h += 1

    # Penalty for first block
    if not found_first_left:
        h += 1

    return h
    
#<(1,2),(3,4),(5,6)> and goal_blocks: <4,2,5>
start_blocks = "(1,2),(3,4),(5,6)"
goal_blocks = "4,2,5"
init_goal_for_heuristics(goal_blocks)
init_goal_for_search(goal_blocks)
start_state = color_blocks_state(start_blocks)
start_time = time.time()
search_result = search(start_state, advanced_heuristic )
end_time = time.time() - start_time

# runtime
print(end_time)
# solution cost
# 
# {5:(4),3:(2),6:(1,7)}
# {4:(5),2:(3),1:(6),7:(6)}
# 
print((search_result))
print("-----------------")
for node in search_result:
    print(node.state.get_state_str())

goal state after init: 4
goal state after init: 2
goal state after init: 5
0.0005958080291748047
[<__main__.search_node object at 0x000002288841B110>, <__main__.search_node object at 0x000002288E19C590>, <__main__.search_node object at 0x000002288E417530>, <__main__.search_node object at 0x000002288E4255B0>, <__main__.search_node object at 0x000002288E424D10>, <__main__.search_node object at 0x000002288E4245F0>]
-----------------
(1,2),(3,4),(5,6)
(2,1),(3,4),(5,6)
(2,1),(4,3),(5,6)
(5,6),(4,3),(2,1)
(5,6),(2,1),(4,3)
(4,3),(2,1),(5,6)


In [21]:
from mcp.server.fastmcp import FastMCP
import math

# Initialize FastMCP
mcp = FastMCP("Unified Solver")

@mcp.tool()
def calculate_sum(a: float, b: float) -> float:
    """Calculates the sum of two numbers."""
    return a + b

@mcp.tool()
def calculate_power(base: float, exponent: float) -> float:
    """Calculates the power of a base number."""
    return math.pow(base, exponent)

# TO DO: Add more tools as needed for your application

@mcp.tool()
def find_the_needed_flips_and_spins(blocks_str: str, goal_blocks: str) -> str:
    """Calculates the needed flips and spins to reach the goal state. For the 'Color Block Tower.' The format of block_str is (x,x),(x,x)... where x can be a numbers and it is the initial state of the problem . the goal state is passed using the goal_blocks param where it is a string of numbers representing the desired visible colors in order seperated with ','."""
    #print(f"find_the_needed_flips_and_spins called with blocks_str: <{blocks_str}> and goal_blocks: <{goal_blocks}>")
    init_goal_for_search(goal_blocks)
    init_goal_for_heuristics(goal_blocks)
    start_state = color_blocks_state(blocks_str)
    search_result = search(start_state, advanced_heuristic )
    if search_result is None:
        return "No solution found"
    steps = []
    for node in search_result:
        steps.append(node.state.get_state_str())
    return " -> ".join(steps)





## 2. LLM + MCP

### 2.1. Global instance of our LLM

In [22]:
import os
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI


# Choose your model here, can be Ollama or Google Gemini
# Can also switch between different model sizes as needed
model = "qwen3:0.6b"
# model = "ibm/granite4:350m"
global_llm = ChatOllama(model=model, temperature=0.0)

# SETUP API KEY if using Google Gemini
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE"

# model = "gemini-2.5-flash"
# model = "gemini-2.5-flash-lite"
# global_llm = ChatGoogleGenerativeAI(model=model, temperature=0)


### 2.2. Our agent graph

In [23]:
from langgraph.graph import MessagesState, START, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver # Optional: For saving graph state


def create_agent_graph(sys_msg, tools):
    """ Creates a LangGraph StateGraph with the given tools integrated."""

    llm = global_llm
    #print(f"Using tools: {tools} , with prompt system message: {sys_msg} ")
    
    if tools.__len__() > 0:
        llm_with_tools = llm.bind_tools(tools)
    else:
        llm_with_tools = llm

    # Node
    def assistant(state: MessagesState):
        #print("Assistant invoked with messages:", state["messages"])
        #print("Assistant state:", state)
        return {
            "messages": [
                llm_with_tools.invoke([sys_msg] + state["messages"], think=False)
            ]
        }

    # Graph
    builder = StateGraph(MessagesState)

    # Define the basic graph structure
    builder.add_node("assistant", assistant)
    builder.add_edge(START, "assistant")

    if tools:
        builder.add_node("tools", ToolNode(tools))  
        builder.add_conditional_edges(
            "assistant",
            tools_condition,
        )
        builder.add_edge("tools", "assistant")

    react_graph = builder.compile()

    return react_graph


async def run_agent(prompt, tools, sys_msg=""):

    sys_msg = SystemMessage(content=sys_msg)

    # 3. Create Graph
    graph = create_agent_graph(sys_msg, tools)
    #print("Graph created with tools:", tools, " and system message:", sys_msg,"and prompt:",prompt)
    # 4. Run (using ainvoke for async tools)
    config = {"configurable": {"thread_id": "1"}}
    result = await graph.ainvoke({"messages": [HumanMessage(content=prompt)]}, config)

    #print("Full result:", result)

    last_msg = result["messages"][-1].content

    #print("last message:", last_msg)
    # Extract tool names and outputs
    tools_used = []
    tools_output = []
    
    # Parsing logic specific to your request
    for msg in result["messages"]:
        # In LangChain, tool calls are usually in 'tool_calls' attribute of AIMessage
        # or 'name' attribute if it is a ToolMessage
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
             for tool_call in msg.tool_calls:
                tools_used.append(tool_call['name'])
        
        if msg.type == 'tool':
            tools_output.append(msg.content)

    return last_msg, tools_used, tools_output



### 2.3. Tools that run spacific agent (with tools and without)


@mcp.tool()
async def ask_agent_with_tools(prompt:str) -> str:
    """ Runs the agent with access to tools.The input MUST be a simple string."""
    tools = [find_the_needed_flips_and_spins]
    #print("Tools being used in ask_agent_with_tools:", tools)
    #print("prompt:", prompt)
    sys_msg=""""
        you are an expert AI specialized in solving the "Color Block Tower" . Your goal is to find the most efficient path to the goal state.
        You have access to the tool 'find_the_needed_flips_and_spins' which can help you find the needed flips and spins to reach the goal state.
        The input will be provided in the format: 'from '(x,y),(z,e)' to 'e,x'' where the first part is the initial tower state and the second part is the goal state.
        Use the tool to find the needed flips and spins to reach the goal state.
        The tool 'find_the_needed_flips_and_spins' takes two parameters:
            blocks_str: str - The initial tower state in the format '(x,y),(z,e)...' where x,y,z,e are numbers or chars.
            goal_blocks: str - The goal state in the format 'e,x,...' where e,x are numbers or chars representing the desired visible colors in order (left side of each pair).
        Output Format:
            list of states separeted with '->' representing each step from initial state to goal state.
        You must only use the tool to find the needed flips and spins.
        
        Provide a step-by-step list of actions taken to reach the goal state from the initial state and determine if it is a spin or flip.
        Output Requirements:

            List the actions in order (e.g., "Step 1: Spin cube at index 2").

            Show the resulting tower state after each action.

            Confirm that the final state's visible colors match the goal.
    """
    results = await run_agent(prompt, tools, sys_msg)
    #print("the result of prompting with tools :",results)
    return results[0]

@mcp.tool()
async def ask_agent_without_tools(prompt:str) -> str:
    """ This agent runs without access to tools.and is specialized in solving the "Color Block Tower" problem.The input MUST be a simple string on format: 'from '(x,y),(z,e)' to 'e,x'' ."""
    # return await run_agent(prompt, [])[0]
    #print("Running agent without tools")
    sys_msg="""
        You are an expert AI specialized in solving the "Color Block Tower" . Your goal is to find the most efficient path to the goal state.
        Rules of the Color Block Tower:
        The tower is a list of cubes. Each cube is a tuple (Visible Color, Hidden Color)3. The first tuple is the Top of the tower, and the last is the Bottom4.
        * Goal: Reach a state where the sequence of "Visible Colors" (the first number in each tuple) matches the goal string exactly5.
        * Action 1 - Spin: Rotate a single cube 90 degrees. This swaps its Visible and Hidden colors 
            (e.g.,Assume that A,B,C,D,E,F,G is the input numbers) (A, B) becomes (B, A)), but its position in the tower stays the same.
        * Action 2 - Flip: The robot grabs the bottom-most cube and one other cube at index i. It reverses the entire segment between them.
            for example, (Assume that A,B,C,D,E,F,G is the input numbers) flipping at index 2 in [(A,B),(C,D),(E,F),(G,H),...] results in [(E,F),(C,D),(A,B),(G,H)].
        * Crucial Mechanic: Because the robot "flips" the segment, the order of cubes is reversed, AND each cube in that segment is rotated (Visible and Hidden colors swap).
        * You cant perform any other actions beyond Spin and Flip.
        * Input Format: The input will be provided in the format: 'from '(x,y),(z,e)' to 'e,x'' where the first part is the initial tower state and the second part is the goal state.
        * Output Format: Provide a step-by-step list of actions taken to reach the goal state from the initial state.
        * Efficiency: Aim to minimize the number of actions taken to reach the goal state.
        * Do not change the pairs of colors in the blocks.
        * Do not duplicate states.
        * Do not make any assumptions beyond the provided rules.
        
        Output Requirements:

            List the actions in order (e.g., "Step 1: Spin cube at index 2").

            Show the resulting tower state after each action.

            Confirm that the final state's visible colors match the goal.
            

    """

    
    results = await run_agent(prompt, [], sys_msg)
    #print("the result of prompting without tools :",results)
    return results[0]

response, tools, outputs = await run_agent("give me the efficient path to solve the Color Block Tower problem from '(1,2),(3,4),(5,6)' to '2,4,5'", [ask_agent_with_tools ],"You have assistant 'ask_agent_with_tools' to find the most efficient path for the Color Block Tower problem.")
#print("Final response:", response)
#print("Tools used:", tools)
#print("Tools outputs:", outputs)



on this code i have explained the game precisely on the "without_tools..."
add game_rules before the two function and for each one keep what it should do and add the game_rules to this and pass it to run_agent

In [40]:
# Detailed Game Rules based on assignment requirements
GAME_RULES = """
Rules of the Color Block Tower:
1. State Representation: The tower is a list of cubes. Each cube is a tuple (Visible Color, Hidden Color). 
   The first tuple in the list is the TOP of the tower (left side), and the last is the BOTTOM (right side).
2. Goal: Reach a state where the sequence of "Visible Colors" (the first number in each tuple) matches the goal string exactly.
3. Action 1 - Spin: Rotate a single cube 90 degrees. This swaps its Visible and Hidden colors (e.g., (A, B) becomes (B, A)), but its position in the tower stays the same.
4. Action 2 - Flip: The robot grabs the bottom-most cube and one other cube at index i. It reverses the entire segment between them.
    * when you flip , you should flip all the tuples from index i to the end of the list. 
   * Crucial Mechanic: Because the robot "flips" the segment, the order of cubes in that segment is reversed. 
5. Cost: Each action (Spin or Flip) has a cost of 1. The goal is to find the minimum sequence of actions.
"""

@mcp.tool()
async def ask_agent_with_tools(prompt:str) -> str:
    """ Runs the agent with access to tools. The input MUST be a simple string."""
    tools = [find_the_needed_flips_and_spins]
    
    sys_msg = f"""
        You are an expert AI specialized in solving the "Color Block Tower".
        {GAME_RULES}
        
        You have access to the tool 'find_the_needed_flips_and_spins' which uses an A* algorithm to find the optimal path.
        You will got the input prompt that contains the initial state and the goal state. 
        convert the input prompt to the two parameters needed for the tool 'find_the_needed_flips_and_spins' and call it.
        
        The tool 'find_the_needed_flips_and_spins' takes two parameters:
            1.blocks_str: str - The initial tower state in the format '(x,y),(z,e)...' where x,y,z,e are numbers or chars.
            2.goal_blocks: str - The goal state is a sequance of numbers separeted with ',' that representing the desired visible colors in order (left side of each pair).
        and returns the sequence of states from the initial to the goal (as string = state1 -> state2 -> ... -> goal_state , each state represented as '(x,y),(z,e)... where x,y,z,e are numbers').
        
        
        Your Task:
        1. Use the tool 'find_the_needed_flips_and_spins' to get the state sequence.
        2. Convert those states into a clear list of actions (Spin at index X or Flip at index Y).
        3. Present the actions and resulting states clearly.
        4. Do not attempt to solve the problem internally; rely solely on the tool for the solution.
        5. Do not change the color pairs within a block; only their visibility or position.
        6. Do not add any states that are not in the tool output.
        Output Requirements:
        - List the actions in order (e.g., "Step 1: Spin cube at index 2").
        - Show the resulting tower state after each action.
        - Confirm that the final state's visible colors match the goal.
    """
    results = await run_agent(prompt, tools, sys_msg)
    return results[0]

@mcp.tool()
async def ask_agent_without_tools(prompt:str) -> str:
    """ This agent runs without access to tools and is specialized in solving the "Color Block Tower" problem. """
    
    sys_msg = f"""
        You are an expert AI specialized in solving the "Color Block Tower".
        {GAME_RULES}
        
        Your Goal: Find the most efficient path to the goal state using internal reasoning.
        Flip and Spin the blocks as needed to reach the goal.
        Flip : Reverse all the blocks from index i to the end of the list (i.e., swapping the block at index i with the block at the end of the list, swapping the block at index i+1 with the block at the end of the list - 1, and so on).
        spin : swap the visible and hidden colors of the block at index i (left with right on the same block).
        Constraints:
        - Efficiency: Aim to minimize the number of actions taken.
        - Consistency: Do not change the color pairs within a block; only their visibility(on spin the same block) or position(when flipping a segment) .
        - Input Format: 'from '(x,y),(z,e)' to 'e,x''
        
        Output Requirements:
        - List the actions in order (e.g., "Step 1: Flip starting at index 1").
        - after each action, show the resulting tower state.
        - Show the resulting tower state after each action.
        - the twoer should not change the color pairs within a block; only their visibility or position.
        - Confirm that the final state's visible colors match the goal.
    """
    results = await run_agent(prompt, [], sys_msg)
    return results[0]
response, tools, outputs = await run_agent("give me the efficient path to solve the Color Block Tower problem from (1,2),(3,4),(5,6),(7,8) to 8,4,2,5", [ask_agent_without_tools ],"Use the assistant 'ask_agent_without_tools' to find the most efficient path for the Color Block Tower problem. return the result of the assistant directly as your final answer.")
print("Final response:", response)
print("Tools used:", tools)
print("Tools outputs:", outputs)




[01/16/26 19:16:31] WARNING  Tool already exists: ask_agent_with_tools                           ]8;id=177877;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mcp\server\fastmcp\tools\tool_manager.py\tool_manager.py]8;;\:]8;id=137179;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mcp\server\fastmcp\tools\tool_manager.py#70\70]8;;\

                    WARNING  Tool already exists: ask_agent_without_tools                        ]8;id=521976;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mcp\server\fastmcp\tools\tool_manager.py\tool_manager.py]8;;\:]8;id=878816;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mcp\server\fastmcp\tools\tool_manager.py#70\70]8;;\

[01/16/26 19:16:32] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=424611;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=978316;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py#1025\1025]8;;\

Final response: The most efficient path to solve the Color Block Tower problem from (1,2),(3,4),(5,6),(7,8) to (8,4,2,5) is as follows:

1. Start at (1,2).
2. Move to (3,4).
3. Move to (5,6).
4. Move to (7,8).
5. Move to (8,4).
6. Move to (2,5).

This path ensures the shortest possible movement while covering all points.
Tools used: []
Tools outputs: []


## 3. Run the Test

In [42]:
# THE JUDGE AGENT RUNNER

sys_msg = """
    You are the Research Supervisor for the Color Block Tower project. 
    You have two assistants:
    1. 'ask_agent_with_tools': Uses a precise A* search algorithm tool to find the mathematically optimal path.
    2. 'ask_agent_without_tools': without any tools.

    Your Goal:
    1. Call 'ask_agent_with_tools' with the user's tower problem.
    2. Call 'ask_agent_without_tools' with the same problem.
    if the 'ask_agent_without_tools' do not try to find solution, note this in your report.
    3. Compare their results. Specifically:
       - Compare the total cost (number of steps).
       - Check if both solutions are valid according to the game rules.
       - Explain any discrepancies (e.g., if the theorist found a sub-optimal path or if the mathematician was more efficient).
    To check validity, ensure that:
       - Each action (Flip or Spin) is correctly applied.
       - The final visible colors match the goal state.
       - flip action is applied correctly (reversing the segment from index i to the end - where i is the flip start index , flipping i with the last index, i+1 with the last index - 1, and so on ).
       - flip should not change the color pairs within a block; 
       - spin action is applied correctly (swapping the visible and hidden colors of the specified block).
       - spin change only the visibility of the colors in the same block.
    Present two results from different assistants in a clear, structured report. compare their approaches, efficiency, and correctness.
    """
    
prompt = "Compare the two assistants for the question 'give me the efficient path to solve the Color Block Tower problem from (1,2),(3,4),(5,6),(7,8) to 8,4,2,5'. ."

tool_list = [ask_agent_with_tools, ask_agent_without_tools]

response, tools, outputs = await run_agent(prompt, tool_list, sys_msg)
print(f"Response: {response}")
print(f"Tools Used: {tools}")
print(f"Tool Outputs: {outputs}")


[01/16/26 19:19:50] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=59406;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=128357;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py#1025\1025]8;;\

[01/16/26 19:19:51] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=284975;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=634419;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py#1025\1025]8;;\

[01/16/26 19:19:58] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=713882;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=407461;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py#1025\1025]8;;\

goal state after init: 8
goal state after init: 4
goal state after init: 2
goal state after init: 5


                    INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=261203;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=718755;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py#1025\1025]8;;\

[01/16/26 19:20:03] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=333918;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=25593;file://C:\Users\bhaah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\httpx\_client.py#1025\1025]8;;\

Response: ### Final Answer:

**Step 1: Flip starting at index 1**  
**Step 2: Flip starting at index 2**  
**Step 3: Flip starting at index 3**  
**Step 4: Flip starting at index 4**  
**Step 5: Flip starting at index 5**

**Final State:**  
(2,1), (3,5), (5,2), (8,4) ✅

### Comparison:

- **Total Cost (Number of Steps):**  
  5 steps (Flip, Flip, Flip, Flip, Flip).

- **Validity Check:**  
  - Each action is correctly applied (Flip reverses the segment from index i to the end, and Spin swaps the visible and hidden colors of the specified block).  
  - The final visible colors match the goal state: **(8,4), (2,5)**.

- **Discrepancy:**  
  - The assistant with tools (A*) found a valid path with 5 steps, while the assistant without tools (ask_agent_without_tools) did not attempt to find a solution.  

**Conclusion:**  
The assistant with tools (A*) found the optimal path with 5 steps, while the assistant without tools (ask_agent_without_tools) did not attempt to solve the problem.
Tools